## DFS
深度搜索的程序模版：

```python
def func():
    def dfs(idx, path):
        '''
        dfs函数，因为是闭包，可以直接访问外部res
        :param idx: 对于全局来说，dfs所在的位置
        :param path: 目前为止dfs已探索的路径
        :return:
        '''
        # path option，对path进行操作，一般是加长路径
        dfs(idx + 1, path)    # 若path是地址引用，需要改成path[:]
        # path recovery，path还原，不影响下一次dfs

    res = None
    init_path = None  # 初始探索路径，一般为0，空字串或空列表
    dfs(0, init_path)
    return res
```

个人习惯把```dfs(idx, path)```写在外部函数中构成闭包，这样做的好处是可以减少```dfs```函数的参数量。需要注意的有两点：
- 如果要在```dfs```函数中修改```res```的话，```res```必须是一个地址引用，如```list```；
- 若```path```是一个地址引用，在递归调用```dfs```时需要传入一个副本```path[:]```。

[Target Sum](https://leetcode.com/problems/target-sum/)。给一个长度为$n$的非负数组，以及一个目标值$S$，在数组的所有元素前选择加上```+```或```-```，使得数组和为$S$。求有多少种配置。

思路：DFS，对每一个位置都需要尝试两种情况，```+```号或```-```号，该方法超时。另一种解法是DP解法，见DP相关文件。

In [23]:
def findTargetSumWays(nums, S: int) -> int:
    res = 0
    if not nums:
        return res

    n = len(nums)

    def dfs(idx, path):
        nonlocal res
        if idx == n:
            if path == S:
                res += 1
            return

        dfs(idx+1, path+nums[idx])
        dfs(idx+1, path-nums[idx])

    init_path = 0
    dfs(0, init_path)
    return res

6162

[Word Ladder](https://leetcode.com/problems/word-ladder/)。给定一个源单词与目标单词还有一个词典，每次允许变换一个字母，求从源单词变为目标单词需要的步数。

思路：BFS，每次改变源单词的一个字母，找到对应的字典中的词，并维护一个访问状态。

In [9]:
def ladderLength(beginWord: str, endWord: str, wordList) -> int:
    word_set = set(wordList)
    visited = set()
    q = list()
    q.append((beginWord, 0))    # 将word与step组成二元组

    while q:
        cur_word, step = q.pop(0)

        if cur_word == endWord:
            return step
        if cur_word in visited:
            continue
        visited.add(cur_word)

        for ch_idx in range(len(cur_word)):
            for i in range(26):
                trans_word = cur_word[:ch_idx]+chr(97+i)+cur_word[ch_idx+1:]
                if trans_word in word_set:
                    q.append((trans_word, step+1))

    return -1

4

[Surrounded Regions](https://leetcode.com/problems/surrounded-regions/)。给一个由'X'和'O'构成的矩阵，将所有被'X'包围的'O'转换成'X'。

思路：DFS转换，但是需要注意的是边界的'O'不会被转换，所以需要特殊处理。分为两步，首先将所有与边界相通的'O'转换成一个特殊字符，那么剩下未被转换的'O'肯定是被'X'包围的，将这些'O'全转换成'X'，再将特殊字符转换成'O'。

In [18]:
def solve(board) -> None:
    if not board or not board[0]:
        return
    rows, cols = len(board), len(board[0])

    def trans_rec(row, col):
        if board[row][col] == 'O':
            board[row][col] = '#'
            if row > 0 and board[row-1][col] == 'O':
                trans_rec(row-1, col)
            if row < rows-1 and board[row+1][col] == 'O':
                trans_rec(row+1, col)
            if col > 0 and board[row][col-1] == 'O':
                trans_rec(row, col-1)
            if col < cols-1 and board[row][col+1] == 'O':
                trans_rec(row, col+1)

    # 将跟边缘连接的'O'保护起来
    for row in range(rows):
        for col in range(cols):
            if (board[row][col] == 'O' and (row == 0 or col == 0 or row == rows-1 or col == cols-1)):
                trans_rec(row, col)

    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'O':
                board[row][col] = 'X'
            if board[row][col] == '#':
                board[row][col] = 'O'

[['X', '#', 'X', '#', 'X', '#'], ['#', 'X', 'O', 'X', 'O', 'X'], ['X', 'O', 'X', 'O', 'X', '#'], ['#', 'X', '#', 'X', '#', 'X']]


[['X', 'O', 'X', 'O', 'X', 'O'],
 ['O', 'X', 'X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'O'],
 ['O', 'X', 'O', 'X', 'O', 'X']]

[Restore IP Addresses](https://leetcode.com/problems/restore-ip-addresses/)。给定一只含数字的字符串，求其能生成的所有合法IP地址。

思路：求出所有可可能结果，DFS，需要两个变量在递归中传递，一个是最重的返回结果```res```，另一个是存储临时结果的```tmp_res```，只有当```tmp_res```完全合格时才会加入到```res```中。该题中有一个隐含条件，IP字段范围是$[0,255]$且不能以$0$开头。

In [12]:
def restoreIpAddresses(s: str):
    res = list()
    if len(s) > 12:
        return res

    tmp_res = list()

    def dfs(s, tmp_res, res):
        # 合法res的最大字段数为4，每个字段最多为3，s大于该长度就直接返回
        thresh = (4-len(tmp_res))*3
        if len(s) > thresh:
            return

        if not s and len(tmp_res) == 4:
            res.append('.'.join(tmp_res))

        # 遍历所有可能的分割，0|1|2|，数字为索引，竖线为切割位置
        for i in range(1, 4):
            if i <= len(s):    # 不要越界
                # 当数字不以0开头且小于等于255，则可以加入tem_res继续遍历
                cur_num = int(s[:i])
                if str(cur_num) == s[:i] and cur_num <= 255:
                    dfs(s[i:], tmp_res+[s[:i]], res)

    dfs(s, tmp_res, res)
    return res

['255.255.11.135', '255.255.111.35']

[Open the Lock](https://leetcode.com/problems/open-the-lock/)。四位旋转密码锁，其中有几个位置是死区(deadend)，一旦转到该位置锁会锁死。给定一系列死区，初始状态为```0000```，问开锁需要转几次旋钮。

思路：BST，用队列实现。

In [18]:
def openLock(deadends, target: str) -> int:
    lock = True
    deadends = set(deadends)
    visited = set()
    q = [('0000', 0)]    # (stat, cnt)

    while q:
        stat, cnt = q.pop(0)

        if stat == target:
            return cnt
        elif stat in visited or stat in deadends:
            continue
        else:
            visited.add(stat)
            for i in range(4):
                plus1 = stat[:i]+str((int(stat[i])+1) % 10)+stat[i+1:]
                q.append((plus1, cnt+1))
                minus1 = stat[:i]+str((int(stat[i])-1) % 10)+stat[i+1:]
                q.append((minus1, cnt+1))

    return -1

[Unique Binary Search Trees](https://leetcode.com/problems/unique-binary-search-trees/)。给定长度为$n$的正数序列，求其能构成多少种不同的BST。

思路：遍历以每一个数字做为根节点，当以数字$i$作为根节点时，那么$[1,i-1]$构成左分支，$[i+1,n]$构成右分支，当前BST的数量为左右分支的可能组合数。设$k$个连续数字可构成$dp[k]$种BST，那么当有$n$个节点时，遍历所有数字分别充当根节点，假设以$i$为根节点，那么其左分支有$i-1$个节点，右分支有$n-i$个节点，那么当前的BST数量为$dp[i-1]*dp[n-i]$。

In [ ]:
def numTrees(n: int) -> int:
    if n < 2:
        return n

    dp = [0 for _ in range(n+1)]
    dp[0] = 1    # 因为是乘法，不能设为0
    dp[1] = 1

    for k in range(2, n+1):    # k: 2->n
        for i in range(1, k+1):    # 遍历所有可能的根节点
            dp[k] += dp[i-1]*dp[k-i]

    return dp[-1]

[Unique Binary Search Trees II](https://leetcode.com/problems/unique-binary-search-trees-ii/)。给定长度为$n$的正数序列，求所有可能的BST结构。

思路：求所有解，DFS。初始化一个有序的正数序列，依次以每一个数作为根节点，BST的性质决定了其左边的数字构成左分支，右边的数字构成右分支。基准情形：范围矛盾则返回空节点，范围大小只有$1$则返回单节点。否则遍历所有可能的根节点，递归求出左分支的答案与右分支的答案，然后进行左右一一组合。

In [ ]:
def generateTrees(n: int):
    def rec(start, end):
        if start > end:
            return [None]    # 这里必须返回[None]
        if start == end:
            return [TreeNode(start)]

        res = list()
        for val in range(start, end+1):
            left_res = rec(start, val-1)
            right_res = rec(val+1, end)

            for left_branch in left_res:
                for right_branch in right_res:
                    root = TreeNode(val)
                    root.left = left_branch
                    root.right = right_branch

                    res.append(root)

        return res

    return rec(1, n) if n >= 1 else list()

[Subsets II](https://leetcode.com/problems/subsets-ii/)。给定一可能有重复元素的数组，求其所有子集。

思路：可能存在重复元素，那么关键就在于避免重复组合。将数组排序后，每次可判断当前新加入的数字是否与上轮数字相同，相同则跳过即可。

In [1]:
def subsetsWithDup(nums):
    n = len(nums)
    nums.sort()
    res = list()

    def rec(idx, path):
        '''
        idx: 相对起点
        path: 已探索路径
        '''
        res.append(path)
        if idx == n:
            return
        for i in range(idx, n):
            if i > idx and nums[i] == nums[i-1]:
                continue
            else:
                path.append(nums[i])
                rec(i+1, path[:])
                path.pop()

    rec(0, list())
    return res

[Increasing Subsequences](https://leetcode.com/problems/increasing-subsequences/)。给一数组，求出所有可能的递增子序列，序列长度至少为$2$。

思路：从左往右DFS，难点在于重复值的处理。在每一个dfs函数中，维护一个集合记录已经出现过的数字。该题与上题还有一个区别，就是进行递归时需要满足递增这个条件。

In [14]:
def findSubsequences(nums):
    res = list()
    n = len(nums)
    if n < 2:
        return res

    def dfs(idx, path):
        if len(path) > 1:
            res.append(path)
        if idx == n:
            return

        seen = set()    # 跳过重复元素
        for i in range(idx, n):
            if i > idx and nums[i] in seen:
                continue
            else:
                # 若path为空，可直接加入；不为空，则需要满足递增
                if not path or nums[i] >= path[-1]:
                    path.append(nums[i])
                    seen.add(nums[i])
                    dfs(i+1, path[:])    # 传入副本，不要更改外部的path
                    path.pop()
                else:
                    continue

    dfs(0, list())
    return res

[[4, 6], [4, 6, 7], [4, 6, 7, 7], [4, 7], [4, 7, 7], [6, 7], [6, 7, 7], [7, 7]]

[Combination Sum III](https://leetcode.com/problems/combination-sum-iii/)。给定$[1,9]$的数字，取出$k$个数使其加和等于$n$，求所有可能的配置。

思路：DFS。

In [5]:
def combinationSum3(k: int, n: int):
    res = list()

    def rec(start, target, path):
        '''
        start: 加到的位置
        target: 目标值
        path: 已保存路径
        '''
        if len(path) == k:
            if target == 0:
                res.append(path)
            return

        for num in range(start, 10):
            if num <= target:
                rec(num+1, target-num, path+[num])
            else:
                break

    rec(1, n, list())
    return res

[[1, 2, 4]]

[Partition to K Equal Sum Subsets](https://leetcode.com/problems/partition-to-k-equal-sum-subsets/)。给一数组与一个正整数$k$，判断该数组能否被划分成和相等的$k$组数。

[Matchsticks to Square](https://leetcode.com/problems/matchsticks-to-square/)。给若干火柴棍，以数组表示，每个元素表示火柴棍的长度，判断这些火柴棍能否拼成一个正方形。

思路：虽然该题并不需要求出一个配置，但是还是通过DFS来做。首先易得每一组的和为$sum/k$，若数组中存在比该值大或者除不尽的情况则直接返回False。将数组排序，并初始化$k$个空数组，由大往小开始填充。第二题是第一题$k=4$时的特殊情况。

In [3]:
def canPartitionKSubsets(nums, k: int) -> bool:
    target, rem = divmod(sum(nums), k)
    if rem:
        return False    # 除不尽则返回False

    nums.sort()
    if nums[-1] > target:
        return False
    while nums and nums[-1] == target:    # 如果有单独的值可以成组，则k减少1
        nums.pop()
        k -= 1

    init_groups = [0]*k    # 初始空组，每组和均为0

    def dfs(groups):
        if not nums:
            return True

        cur_num = nums.pop()    # 弹出当前最大值
        for i in range(len(groups)):    # 尝试将cur_num加到所有组上去
            if groups[i]+cur_num <= target:
                groups[i] += cur_num
                if dfs(groups):    # 如果所有子情况都满足，那么返回True
                    return True
                groups[i] -= cur_num

        nums.append(cur_num)    # 走到这一步肯定是失败，将数组复原
        return False

    return dfs(init_groups)

True

[Binary Tree Paths](https://leetcode.com/problems/binary-tree-paths/)。给一个二叉树，返回所有根节点到叶节点的路径。

思路：DFS。

In [ ]:
def binaryTreePaths(root: TreeNode):
    res = list()
    if not root:
        return res

    path = ''

    def dfs(cur_node, path):
        if not path:
            path += str(cur_node.val)
        else:
            path += '->{}'.format(cur_node.val)

        if not cur_node.left and not cur_node.right:    # 叶节点判定
            res.append(path)
            return
        if cur_node.left:
            dfs(cur_node.left, path)
        if cur_node.right:
            dfs(cur_node.right, path)

    dfs(root, path)
    return res

[Smallest String Starting From Leaf](https://leetcode.com/problems/smallest-string-starting-from-leaf/)。给一颗字符二叉树，求一条从叶结点到根节点且字典序最小的路径。规定在同样前缀的情况下，字符越短字典序越小。

思路：DFS。路径由叶节点到根节点，那么路径字串的首字符由叶节点决定，所以在走到叶节点之前是无法确定字典序的。每当DFS走到叶节点，比较已有结果与当前路径的字典序。

In [ ]:
def smallestFromLeaf(root: TreeNode) -> str:
    if not root:
        return ''

    res = '{'    # 大括号的ASCII码大于所有字母
    path = list()

    def dfs(vis_node, path):
        path.append(chr(vis_node.val+97))    # 'a':97
        if not vis_node.left and not vis_node.right:
            path_s = ''.join(reversed(path))
            res = min(res, path_s)
        if vis_node.left:
            dfs(vis_node.left, path)
        if vis_node.right:
            dfs(vis_node.right, path)

        path.pop()    # 注意参数path是一个list，需要还原

    dfs(root, path)
    return res

[Sum Root to Leaf Numbers](https://leetcode.com/problems/sum-root-to-leaf-numbers/)。给一颗二叉树，求所有根节点到叶节点路径的和。

思路：DFS。

In [ ]:
def sumNumbers(root: TreeNode) -> int:
    res = 0
    if not root:
        return 0

    path = 0

    def dfs(vis_node, path):
        path = path*10+vis_node.val
        if not vis_node.left and not vis_node.right:
            res += path

        if vis_node.left:
            dfs(vis_node.left, path)
        if vis_node.right:
            dfs(vis_node.right, path)

        path -= vis_node.val

    dfs(root, path)
    return res

[Binary Tree Right Side View](https://leetcode.com/problems/binary-tree-right-side-view/)。给一颗二叉树，求右视图所看到的节点序列。

思路：树的层次遍历(BST)，维护一个层级节点数即可，每访问到当前层的最后一个节点就加入返回列表。

In [ ]:
def rightSideView(root: TreeNode):
    res = list()
    if not root:
        return res

    q = [root]
    while q:
        level_size = len(q)
        for idx in range(level_size):
            vis_node = q.pop(0)
            if idx == level_size-1:
                res.append(vis_node.val)

            if vis_node.left:
                q.append(vis_node.left)
            if vis_node.right:
                q.append(vis_node.right)

    return res

[Minesweeper](https://leetcode.com/problems/minesweeper/)。扫雷模拟，'M'表示地雷，'E'表示未点击方块，'B'表示空白块，$[1-8]$的数字表示周围的地雷数，'X'表示踩雷。给一个游戏盘与一个点击位置，返回点击后的状态。

思路：逻辑实现，递归。如果点击到'B'与数字，无改变；踩到雷，改成'X'；点击'E'，若周围没有雷，则递归点击周围所有格子，否则改成数字。难点在于周围八个点的存在性判断，最蠢的方法就是写多个判断条件。

In [7]:
def updateBoard(board, click):
    rows, cols = len(board), len(board[0])
    int_l = ['1', '2', '3', '4', '5', '6', '7', '8']

    def bfs(row, col):
        if row < 0 or row >= rows or col < 0 or col >= cols:     # 输入合法性判断
            return

        if board[row][col] == 'M':
            board[row][col] = 'X'
            return

        if board[row][col] == 'B' or board[row][col] in int_l:
            return

        if board[row][col] == 'E':
            n = 0
            # 首先扫描周围的雷，八个判断
            if row > 0:    # 上
                if board[row-1][col] == 'M':
                    n += 1

            if col > 0:    # 左
                if board[row][col-1] == 'M':
                    n += 1

            if row > 0 and col > 0:    # 左上
                if board[row-1][col-1] == 'M':
                    n += 1

            if row < rows-1:    # 下
                if board[row+1][col] == 'M':
                    n += 1

            if col > 0 and row < rows-1:    # 左下
                if board[row+1][col-1] == 'M':
                    n += 1

            if col < cols-1:    # 右
                if board[row][col+1] == 'M':
                    n += 1

            if row > 0 and col < cols-1:
                if board[row-1][col+1] == 'M':
                    n += 1

            if row < rows-1 and col < cols-1:
                if board[row+1][col+1] == 'M':
                    n += 1

            if n == 0:
                board[row][col] = 'B'
                bfs(row-1, col-1)
                bfs(row-1, col)
                bfs(row-1, col+1)
                bfs(row, col+1)
                bfs(row+1, col+1)
                bfs(row+1, col)
                bfs(row+1, col-1)
                bfs(row, col-1)
            else:
                board[row][col] = str(n)
                return

    bfs(click[0], click[1])
    return board

[['B', '1', 'E', '1', 'B'],
 ['B', '1', 'M', '1', 'B'],
 ['B', '1', '1', '1', 'B'],
 ['B', 'B', 'B', 'B', 'B']]

[01 Matrix](https://leetcode.com/problems/01-matrix/)。给一个只含$0$和$1$的矩阵，返回一个矩阵，该矩阵记录的是相应位置元素离最近$0$元素的距离，距离只算上下左右，不能斜着走。

思路：BFS。因为矩阵中存在$0$元素，并且要求的也是距$0$的距离，那么BFS的起点就是矩阵中的所有$0$元素。

In [25]:
def updateMatrix(matrix):
    rows, cols = len(matrix), len(matrix[0])
    q = list()
    res = [[0x7FFFFFFF for _ in range(cols)] for _ in range(rows)]

    for row in range(rows):
        for col in range(cols):
            if matrix[row][col] == 0:
                q.append((row, col))
                res[row][col] = 0

    while q:
        row, col = q.pop(0)

        if row > 0:
            if res[row-1][col] > res[row][col]+1:
                res[row-1][col] = res[row][col]+1
                q.append((row-1, col))

        if row < rows-1:
            if res[row+1][col] > res[row][col]+1:
                res[row+1][col] = res[row][col]+1
                q.append((row+1, col))

        if col > 0:
            if res[row][col-1] > res[row][col]+1:
                res[row][col-1] = res[row][col]+1
                q.append((row, col-1))

        if col < cols-1:
            if res[row][col+1] > res[row][col]+1:
                res[row][col+1] = res[row][col]+1
                q.append((row, col+1))

    return res

[[0, 0, 0], [0, 1, 0], [1, 2, 1]]

[Friend Circles](https://leetcode.com/problems/friend-circles/)。给一无向图的邻接矩阵，求联通分量数。

思路：无向图的BFS。